# Calculating a seismic attribute with Python Tool Pro

With the existing Python Tool, scripts are written and run from an embedded editor in the Python Tool. In contrast, Python Tool Pro offers an API which is meant to be used in the end user's scripts run in a process separated from Petrel. The end user will use the API of the Python Tool Pro in a Jupyter Notebook, an IPython shell, or on any platform capable of running Python scripts.

This example demonstrates how to use Python Tool Pro to calculate a seismic attribute using a pretrained model shared on github by a 3rd party.


# Loading required libraries

First load the required libraries:

In [ ]:
import math
from tensorflow.keras.models import load_model, model_from_json
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import matplotlib
import numpy as np
import itkwidgets
from blueback.pythontool.grpc.petrelconnection import PetrelConnection
from blueback.pythontool.grpc import petrelinterface_pb2

# Managing GPU memory

Then some housekeeping required to manage my gpu memory. This might not be necessary on your computer:

In [ ]:
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

# Connect to Petrel
This Jupyter Notebook is a client which connects to the Python Tool Pro server. The port number must be given as shown below.

In [ ]:
petrel = PetrelConnection(port=40129)
petrel.open()

print(f'Currently open Petrel project is {petrel.get_current_project_name()}')

# Retrieving data from Petrel
Lets use a ipywidget to make it easier to select one of the seismic cubes in the project.

In [ ]:
cubes = petrel.seismic_cubes
cube_names = list(petrel.seismic_cubes.keys())
import ipywidgets as widgets
w = widgets.Dropdown(
    options=cube_names,
    value=cube_names[0],
    description='Select cube:',
    disabled=False,
)
display(w)

Get a reference to a cube.

In [ ]:
cube = petrel.seismic_cubes[w.value]

Lets double check that the selected seismic is large enough to be processed with the tensorflow model.

In [ ]:
assert cube.extent.i >= 128 and cube.extent.j >= 128 and cube.extent.k >= 128

Get a 128x128x128 chunk of the cube:

In [ ]:
span = 127
x = int((cube.extent.i - 128)/2)
y = int((cube.extent.j - 128)/2)
z = int((cube.extent.k - 128)/2)
arr  = cube.chunk((x,x+span),(y,y+span),(z,z+span)).as_array()

# Visualising seismic chunk
Then use itkwidgets to get a view of the cube:

In [ ]:
itkwidgets.view(image=(np.rot90(arr, 1, (0,2))-arr.mean())/arr.std(),shadow=False, gradient_opacity=0.0, ui_collapsed=True, cmap=itkwidgets.cm.BuRd, opacity_gaussians= [[{'position': 0.5, 'height': 1, 'width': 0.5, 'xBias': 0.0, 'yBias': 2.0}]])

# Processing the seismic chunk

Load the FaultSeg model:

In [ ]:
loaded_model = load_model("fseg-70.hdf5", custom_objects={'cross_entropy_balanced': tf.keras.losses.BinaryCrossentropy()})

FaultSeg model download link can be found in the readme file

Here is a wrapper function that handles applying the model to an array delivered by petrel.

In [ ]:
def fault_attribute_calculator(arr):
    arr = np.rot90(arr, 1, (0,2))
    arr = (arr - arr.mean())/arr.std()
    n1, n2, n3 = 128, 128, 128
    gx = np.reshape(arr,(1,n1,n2,n3,1))
    Y = loaded_model.predict(gx,verbose=1)
    Y = Y.reshape((n1,n2,n3))
    return np.rot90(Y, -1, (0,2))

And apply it to the seismic cube chunk we got before.

In [ ]:
faults = fault_attribute_calculator(arr)

And then visualize the results:

In [ ]:
# The code contained within this cell is modified from the itkwidget source code. It is licenced under the apache-2 licence. Please see the included file "itkwidgets-licence.txt" for details.

import numpy as np
import ipywidgets as widgets
from itkwidgets.widget_viewer import Viewer
from traitlets import CBool
import IPython

def fault_compare(image1, image2,
            link_cmap=False, link_gradient_opacity=False,
            **viewer_kwargs):
    """Compare two images by visualizing them side by side.
    Visualization traits, e.g. the view mode, camera, etc., are linked
    between the viewers. Optional trait linking can be enabled in widget's
    user interface.
    """

    viewer1 = Viewer(image=image1, shadow=False, gradient_opacity=0.0, ui_collapsed=True, cmap=itkwidgets.cm.BuRd, opacity_gaussians= [[{'position': 0.5, 'height': 1, 'width': 0.33, 'xBias': 0.0, 'yBias': 2.0}]])
    # Collapse the second viewer's user interface by default.
    if 'ui_collapsed' not in viewer_kwargs:
            viewer_kwargs['ui_collapsed'] = True
    viewer2 = Viewer(image=image2, shadow=False, gradient_opacity=0.0, ui_collapsed=True, 
                cmap=itkwidgets.cm.inferno,
                opacity_gaussians= [[{'position': 0.5, 'height': 1, 'width': 0.5, 'xBias': 1, 'yBias': 1.75}]])


    widgets.jslink((viewer1, 'mode'), (viewer2, 'mode'))
    widgets.jslink((viewer1, 'camera'), (viewer2, 'camera'))
    widgets.jslink((viewer1, 'roi'), (viewer2, 'roi'))
    widgets.jslink((viewer1, 'rotate'), (viewer2, 'rotate'))
    widgets.jslink((viewer1, 'annotations'), (viewer2, 'annotations'))
    widgets.jslink((viewer1, 'x_slice'), (viewer2, 'x_slice'))
    widgets.jslink((viewer1, 'y_slice'), (viewer2, 'y_slice'))
    widgets.jslink((viewer1, 'z_slice'), (viewer2, 'z_slice'))
    widgets.jslink((viewer1, 'slicing_planes'), (viewer2, 'slicing_planes'))

    link_widgets = []
    link_widgets.append(widgets.Label('Link:'))

    class UpdateLink(object):
        def __init__(self, enable, name):
            self.link = None
            self.name = name
            if enable:
                self.link = widgets.jslink((viewer1, name), (viewer2, name))

        def __call__(self, change):
            if change.new:
                self.link = widgets.jslink((viewer1, self.name), (viewer2, self.name))
            else:
                self.link.unlink()

    link_cmap_widget = widgets.Checkbox(description='cmap', value=link_cmap)
    update_cmap_link = UpdateLink(link_cmap, 'cmap')
    link_cmap_widget.observe(update_cmap_link, 'value')
    link_widgets.append(link_cmap_widget)

    link_gradient_opacity_widget = widgets.Checkbox(description='gradient_opacity', value=link_gradient_opacity)
    update_gradient_opacity_link = UpdateLink(link_gradient_opacity, 'gradient_opacity')
    link_gradient_opacity_widget.observe(update_gradient_opacity_link, 'value')
    link_widgets.append(link_gradient_opacity_widget)

    link_widget = widgets.HBox(link_widgets)

    widget = widgets.AppLayout(header=None,
            left_sidebar=viewer1,
            center=None,
            right_sidebar=viewer2,
            footer=link_widget,
            pane_heights=[1, 6, 1])
    return widget

fault_compare(np.rot90(arr, 1, (0,2)), np.rot90(faults, 1, (0,2)))

# Writing results back to Petrel

We are happy with the result, and want to apply it to the entire cube in petrel.

First lets clone original cube:

In [ ]:
cube_fault_prediction = cube.clone('cube_fault_prediction', copy_values = False)

And define a function that applies the model across the cube:

In [ ]:
a = 0.5
overlapp = 0.1
def apply_calculator(src_cube, dst_cube, calculator, chunk_size = (128,128,128)):
    m1, m2, m3 = src_cube.extent.i, src_cube.extent.j, src_cube.extent.k
    n1, n2, n3 = chunk_size
    
    x_count = math.ceil(2*m1/n1)
    y_count = math.ceil(2*m2/n2)
    z_count = math.ceil(2*m3/n3)
    for x in range(x_count):
        if ( x*(n1/(1+overlapp)) > (m1 - n1/2 -1)):
              continue
        for y in range(y_count):
            if (y*(n2/(1+overlapp)) > (m2 - n2/2 -1)):
                continue
            for z in range(z_count):
                if (z*(n3/(1+overlapp)) > (m3 - n3/2 -1) ):
                      continue
                x_offset = math.floor(min(x*(n1/(1+overlapp)), m1 - n1 -1))
                y_offset = math.floor(min(y*(n2/(1+overlapp)), m2 - n2 -1))
                z_offset = math.floor(min(z*(n3/(1+overlapp)), m3 - n3 -1))
                x_range = (x_offset + 1, x_offset+n1)
                y_range = (y_offset + 1, y_offset+n2)
                z_range = (z_offset + 1, z_offset+n3)
                src_data = src_cube.chunk(x_range, y_range, z_range).as_array()
                with dst_cube.chunk(x_range, y_range, z_range).values() as dst:
                    result = calculator(src_data)
                    mask = (dst == 0)
                    dst[mask] = result[mask]
                    dst[2:-2,2:-2,2:-2] = a*result[2:-2,2:-2,2:-2] + (1-a)*dst[2:-2,2:-2,2:-2]


Note that the cloned seismic cube persists the template of the original seismic cube. This means that in order to see the changes written to it from petrel you will need to change the template to a more typical fault attribute template. The cloned cube will have values in the range 0 to 1 after it has been processed.

Once we have changed the template of the cloned seismic, we are ready to apply our calculator to the cloned cube. We can then see our petrel cube get updated in real time in petrel.

In [ ]:
apply_calculator(cube, cube_fault_prediction, fault_attribute_calculator)